#Download Crop Faces

In [ ]:
%cd /content/WIKI_CROP
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar

In [ ]:
!tar -xvf wiki_crop.tar

#Packages

In [ ]:
# Misc. packages
from google.colab.patches import cv2_imshow
from scipy.io import loadmat
import numpy as np
from numpy import *
import random
import cv2

# Tensorflow
import tensorflow_datasets as tfds
import tensorflow as tf

# Keras
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, adam
from keras.models import Sequential
from keras.utils import np_utils

# scikit tools
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from sklearn import metrics

# Image manipulation
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image


#Gather Training Data

In [ ]:
# Return bin classification
def classify(age):
  if 0 <= age < 20:
    return 0
  elif 20 <= age < 40:
    return 1
  elif 40 <= age < 60:
    return 2
  elif 60 <= age < 80:
    return 3
  elif 80 <= age <= 100:
    return 4



# Key info.
training = []
IMG_SIZE = 132

# Load data
annots = loadmat("/content/WIKI_CROP/wiki_crop/wiki.mat")
wiki = annots['wiki'][0][0]

# Gather training data
start = 0
end = 4000
for i in range(start, end):
  # Resize images
  filename = wiki[2][0][i][0]
  img = cv2.imread("/content/WIKI_CROP/wiki_crop/" + filename)
  new_array = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

  # Real Age
  born = wiki[0][0][i]
  pic_time = wiki[1][0][i]
  real_age = pic_time - int(born / 365)
  classification = classify(real_age)

  # Feed valid data
  if classification is not None and real_age >= 0:
    training.append([new_array, classification])

random.shuffle(training)

# "Shape" of CNN
X =[]
y =[]
for features, label in training:
  X.append(features)
  y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

# Normalize & Categorize
X = X.astype('float32')
X /= 255
Y = np_utils.to_categorical(y, 5)

# 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5)



# Compile & Train

In [ ]:
batch_size = 500
nb_epochs = 5
nb_classes = 5

kernel_size = 3
nb_filters = 32
pool_size = 2

model = tf.keras.Sequential([

    tf.keras.layers.Conv2D(nb_filters * 2, (kernel_size, kernel_size), padding='same', activation=tf.nn.relu,
                           input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D((pool_size, pool_size), strides = 2),

    tf.keras.layers.Conv2D(nb_filters, (kernel_size * 3, kernel_size * 3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((pool_size, pool_size), strides = 2),


    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6,  activation=tf.nn.softmax)
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(np.array(X_train), np.array(y_train), batch_size = batch_size, epochs = nb_epochs, verbose =1, validation_data = (np.array(X_test), np.array(y_test)))

# Accuracy
score = model.evaluate(np.array(X_test), np.array(y_test), verbose = 0)
print("Accuracy: " + str(score[1]))

Epoch 1/5
7/7 [==============================] - 35s 2s/step - loss: 2.5610 - accuracy: 0.4134 - val_loss: 1.7288 - val_accuracy: 0.6137
Epoch 2/5
7/7 [==============================] - 4s 577ms/step - loss: 1.5567 - accuracy: 0.5995 - val_loss: 1.3822 - val_accuracy: 0.6137
Epoch 3/5
7/7 [==============================] - 4s 559ms/step - loss: 1.2490 - accuracy: 0.5724 - val_loss: 1.1900 - val_accuracy: 0.4832
Epoch 4/5
7/7 [==============================] - 4s 579ms/step - loss: 1.1692 - accuracy: 0.5743 - val_loss: 1.1296 - val_accuracy: 0.6137
Epoch 5/5
7/7 [==============================] - 4s 564ms/step - loss: 1.1543 - accuracy: 0.5995 - val_loss: 1.1147 - val_accuracy: 0.6137
Accuracy: 0.6136950850486755
